In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
from datetime import datetime, timedelta
import os

# Import the main functionality from the SimFin Python API.
import simfin as sf #pip install simfin

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

In [2]:
# Boolean whether this is being run under pytest. We will
# use this to make certain parts of the tutorial run faster.
running_pytest = ('PYTEST_CURRENT_TEST' in os.environ)

In [3]:
# SimFin data-directory.
sf.set_data_dir('~/simfin_data/')

In [4]:
# SimFin load API key or use free data.
sf.load_api_key(path='~/simfin_api_key.txt', default_key='free')

In [5]:
# Seaborn set plotting style.
sns.set_style("whitegrid")

# DATA

In [6]:

hub = sf.StockHub(market='us',
                  refresh_days=30,
                  refresh_days_shareprices=1)

# Financial Signals

In [7]:
%%time
df_fin_signals = hub.fin_signals(variant='latest')

Dataset "us-income-ttm" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-balance-ttm" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-ttm" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-latest" on disk (0 days old).
- Loading from disk ... Done!
Cache-file 'fin_signals-899db0b4.pickle' on disk (0 days old).
- Loading from disk ... Done!
Wall time: 21.3 s


In [8]:
df_fin_signals.dropna().head()

,,(Dividends + Share Buyback) / FCF,Asset Turnover,CapEx / (Depr + Amor),Current Ratio,Debt Ratio,Dividends / FCF,Gross Profit Margin,Interest Coverage,Inventory Turnover,Log Revenue,Net Acquisitions / Total Assets,Net Profit Margin,Quick Ratio,R&D / Gross Profit,R&D / Revenue,Return on Assets,Return on Equity,Return on Research Capital,Share Buyback / FCF
Ticker,Date,,,,,,,,,,,,,,,,,,,
A,2020-06-12,0.675676,0.556307,0.675926,3.409660,0.210631,0.224099,0.547121,29.375000,7.639269,9.700617,0.083906,0.222554,2.660107,0.146395,0.080096,0.123808,0.217951,6.830846,0.451577
ABBV,2020-06-12,1.307143,0.575085,0.343390,1.036608,0.654301,0.457830,0.770729,6.157635,19.181551,10.513843,0.037746,0.164180,0.784522,0.412288,0.317763,0.094418,-0.684896,2.425487,0.849313
ABCD,2018-12-27,-0.037860,0.867092,0.817079,0.561277,0.235071,-0.000000,0.725138,7.135607,94.287559,8.205924,0.021274,0.268725,0.431389,0.133771,0.097002,0.233009,-214.800995,7.475457,-0.037860
ABT,2020-06-12,0.475056,0.454415,0.457377,1.581148,0.281822,0.460225,0.583407,5.643284,7.520930,10.487464,0.110649,0.085343,0.944365,0.132950,0.077564,0.038781,0.084230,7.521611,0.014831
ACT,2020-06-12,0.194790,0.560572,0.207777,1.451054,0.462500,-0.000000,0.447330,2.363512,4.610585,9.883576,0.390015,-0.075047,0.619728,0.160051,0.071596,-0.042069,-0.153148,6.247991,0.194790


In [9]:
%%time
df_fin_signals_2y = hub.fin_signals(variant='latest',
                                    func=sf.avg_ttm_2y)

Cache-file 'fin_signals-4a256f6a.pickle' on disk (0 days old).
- Loading from disk ... Done!
Wall time: 256 ms


In [10]:
df_fin_signals_2y.dropna().head()

,,(Dividends + Share Buyback) / FCF,Asset Turnover,CapEx / (Depr + Amor),Current Ratio,Debt Ratio,Dividends / FCF,Gross Profit Margin,Interest Coverage,Inventory Turnover,Log Revenue,Net Acquisitions / Total Assets,Net Profit Margin,Quick Ratio,R&D / Gross Profit,R&D / Revenue,Return on Assets,Return on Equity,Return on Research Capital,Share Buyback / FCF
Ticker,Date,,,,,,,,,,,,,,,,,,,
A,2020-06-12,0.457838,0.546824,0.850218,3.362339,0.224649,0.221747,0.544747,24.339674,7.792699,9.687280,0.045596,0.136383,2.709175,0.143510,0.078183,0.075395,0.134642,6.970978,0.236091
ABBV,2020-06-12,0.951662,0.501064,0.347586,1.117554,0.596013,0.440825,0.761243,8.279215,18.109763,10.492656,0.127148,0.189834,0.841171,0.320335,0.244725,0.093220,0.555525,3.402063,0.510837
ABCD,2018-12-27,-0.036670,0.941183,0.828530,0.536716,0.342381,-0.000000,0.715863,5.936215,78.751252,8.201384,0.028382,0.190733,0.405365,0.126111,0.090350,0.173736,-107.597654,7.958854,-0.036670
ABT,2020-06-12,0.405873,0.427784,0.412296,1.620985,0.295273,0.419400,0.574758,4.483577,7.477794,10.470735,0.055847,0.051039,0.993714,0.136990,0.078701,0.022747,0.049646,7.306160,-0.013527
ACT,2020-06-12,0.095160,0.715048,0.219165,1.510273,0.324199,-0.000000,0.436843,5.579701,5.530521,9.820080,0.224477,-0.023152,0.682453,0.151487,0.066266,-0.008539,-0.054338,6.622415,0.095160


# Growth Signals

In [11]:
%%time
df_growth_signals = hub.growth_signals(variant='latest')

Dataset "us-income-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-balance-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Cache-file 'growth_signals-899db0b4.pickle' on disk (0 days old).
- Loading from disk ... Done!
Wall time: 16.4 s


In [12]:
df_growth_signals.dropna().head()

,,Assets Growth,Assets Growth QOQ,Assets Growth YOY,Earnings Growth,Earnings Growth QOQ,Earnings Growth YOY,FCF Growth,FCF Growth QOQ,FCF Growth YOY,Sales Growth,Sales Growth QOQ,Sales Growth YOY
Ticker,Date,,,,,,,,,,,,
A,2020-06-12,0.027095,0.007819,0.027095,3.713080,-0.638889,-0.112195,0.076364,0.224138,-0.164706,0.063347,-0.035826,0.026534
AA,2020-06-12,-0.066682,0.001129,-0.066682,-1.859155,-5.627907,-2.326667,-0.790464,-0.744186,-6.210526,0.078183,-0.186902,-0.120065
AAL,2020-06-12,0.140897,0.003417,0.140897,-0.245858,-0.420063,-0.005376,3.458763,1.572993,-0.323417,0.040480,-0.032364,0.017594
AAMC,2020-06-12,-0.092099,-0.027731,-0.092099,-0.265901,-0.804196,-0.807516,-5.328671,-9.184165,0.123051,-0.082786,-0.030070,-0.036772
AAN,2020-06-12,0.190863,0.145497,0.190863,-0.313707,-0.091751,0.073345,0.230101,-5.926233,-0.223072,0.112266,0.019049,0.060006


In [13]:
%%time
df_growth_signals_2y = hub.growth_signals(variant='latest',
                                          func=sf.avg_ttm_2y)

Cache-file 'growth_signals-4a256f6a.pickle' on disk (0 days old).
- Loading from disk ... Done!
Wall time: 123 ms


In [14]:
df_growth_signals_2y.dropna().head()

,,Assets Growth,Assets Growth QOQ,Assets Growth YOY,Earnings Growth,Earnings Growth QOQ,Earnings Growth YOY,FCF Growth,FCF Growth QOQ,FCF Growth YOY,Sales Growth,Sales Growth QOQ,Sales Growth YOY
Ticker,Date,,,,,,,,,,,,
A,2020-06-12,0.061452,0.008853,0.061452,1.560148,-1.139757,0.068902,0.172797,0.434650,0.013441,0.077465,-0.019977,0.060454
AA,2020-06-12,-0.032756,-0.009494,-0.032756,0.598994,-3.696607,-1.330000,-2.387741,-0.903139,-6.771930,0.153019,-0.106683,0.021889
AAL,2020-06-12,0.076653,0.006397,0.076653,-0.199626,-0.349566,-0.105252,1.461995,-0.322860,0.255545,0.052629,-0.025569,0.049165
AAMC,2020-06-12,-0.127990,-0.076131,-0.127990,0.309549,0.320996,0.751781,-3.008017,-6.530262,-0.073474,-0.121365,-0.018101,-0.114075
AAN,2020-06-12,0.113440,0.077715,0.113440,0.363056,-0.398754,0.026785,-0.078393,-6.105395,0.339162,0.102433,0.049192,0.095277


# Valuation Signals

In [15]:
%%time
df_val_signals = hub.val_signals(variant='latest')

Cache-file 'val_signals-9541e2e6.pickle' on disk (0 days old).
- Loading from disk ... Done!
Wall time: 162 ms


In [16]:
df_val_signals.dropna().head()

,,Dividend Yield,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/Cash,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales
Ticker,Date,,,,,,,,,,,
A,2020-06-12,0.007144,0.040098,0.031878,2.785658e+10,5.435431,12.926490,24.938751,31.370028,-327.724529,-34.853406,5.550226
AAL,2020-06-12,0.023876,0.183091,0.112242,7.706556e+09,-12.117227,1.772029,5.461769,8.909313,-0.146043,-0.140543,0.172314
AAN,2020-06-12,0.002138,0.065821,0.081020,3.039169e+09,1.675851,24.479029,15.192657,12.342657,17.797483,-5.709288,0.782038
AAP,2020-06-12,0.001766,0.043085,0.064480,9.971472e+09,2.811064,18.557446,23.209982,15.508746,-5.690018,-2.337071,1.032378
AAPL,2020-06-12,0.008706,0.035067,0.036699,1.630310e+12,15.400625,20.355468,28.516883,27.249042,-14.454129,-11.457638,6.307053


In [17]:
%%time
df_val_signals_2y = hub.val_signals(variant='latest',
                                    func=sf.avg_ttm_2y)

Cache-file 'val_signals-2eee516b.pickle' on disk (0 days old).
- Loading from disk ... Done!
Wall time: 135 ms


In [18]:
df_val_signals_2y.dropna().head()

,,Dividend Yield,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/Cash,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales
Ticker,Date,,,,,,,,,,,
A,2020-06-12,0.006821,0.024303,0.030747,2.785658e+10,5.718864,10.784586,41.147097,32.523742,204.077546,-50.984370,5.720625
AAL,2020-06-12,0.024589,0.212936,0.068708,7.706556e+09,-9.318689,1.598871,4.696256,14.554403,-0.158255,-0.151470,0.175732
AAN,2020-06-12,0.002404,0.080865,0.073442,3.039169e+09,1.702759,19.384683,12.366311,13.616167,8.390107,-9.847020,0.823603
AAP,2020-06-12,0.001780,0.046828,0.060893,9.971472e+09,2.811303,16.951468,21.354658,16.422313,-16.574012,-3.261144,1.048771
AAPL,2020-06-12,0.008390,0.033886,0.035124,1.630310e+12,14.009832,19.404758,29.511081,28.470320,-14.597853,-11.628450,6.445098


# Combine Signals

In [19]:
# Combine the DataFrames.
dfs = [df_fin_signals, df_growth_signals, df_val_signals]
df_signals = pd.concat(dfs, axis=1)

# Show the result.
df_signals.head()

,,(Dividends + Share Buyback) / FCF,Asset Turnover,CapEx / (Depr + Amor),Current Ratio,Debt Ratio,Dividends / FCF,Gross Profit Margin,Interest Coverage,Inventory Turnover,Log Revenue,...,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/Cash,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales
Ticker,Date,,,,,,,,,,,,,,,,,,,,,
A,2020-06-12,0.675676,0.556307,0.675926,3.409660,0.210631,0.224099,0.547121,29.375000,7.639269,9.700617,...,0.040098,0.031878,2.785658e+10,5.435431,12.926490,24.938751,31.370028,-327.724529,-34.853406,5.550226
AA,2020-06-12,-0.053892,0.816746,0.554149,1.467713,0.112998,-0.000000,0.241866,17.031746,7.244024,10.115011,...,-0.056208,0.076940,2.170522e+09,0.306441,2.134240,-17.791168,12.997141,-0.456088,-0.339781,0.166553
AAC,2019-11-04,0.000000,0.724773,2.129626,2.117862,0.500277,0.000000,0.402427,-1.627675,NaN,8.458579,...,-2.166485,-3.940417,1.034256e+07,0.066510,1.758939,-0.461577,-0.253780,-0.075938,-0.062823,0.035979
AAL,2020-06-12,1.350289,0.735749,NaN,0.435992,0.395315,0.212717,0.264198,2.777778,26.845138,10.650541,...,0.183091,0.112242,7.706556e+09,-12.117227,1.772029,5.461769,8.909313,-0.146043,-0.140543,0.172314
AAMC,2020-06-12,-1.171567,0.328701,0.407583,13.154065,NaN,0.000000,-0.110731,2.693211,NaN,7.198024,...,-0.300718,-0.123144,2.513318e+07,0.595658,0.663687,-3.325374,-8.120575,0.702515,0.783820,1.593027


In [20]:
mask = (df_signals[P_NETNET] > 0) & (df_signals[P_NETNET] < 1)

In [21]:
mask.head()

Ticker  Date      
A       2020-06-12    False
AA      2020-06-12    False
AAC     2019-11-04    False
AAL     2020-06-12    False
AAMC    2020-06-12     True
Name: P/NetNet, dtype: bool

In [22]:
df_signals.loc[mask, P_NETNET]

Ticker  Date      
AAMC    2020-06-12    0.783820
ALT     2020-06-12    0.933848
AVGR    2020-06-12    0.145914
CGA     2020-06-12    0.058775
CLBS    2020-06-12    0.471950
CLRB    2020-06-12    0.561256
CRVS    2020-06-12    0.960585
CUO     2020-05-19    0.580236
CYCC    2020-06-12    0.202383
CYIG    2020-06-12    0.050105
FPRX    2020-06-12    0.939102
GLYC    2020-06-12    0.702256
GTXI    2020-06-12    0.515962
KKR     2020-06-12    0.689317
NLNK    2020-06-12    0.593927
NSPR    2020-06-12    0.142141
OMED    2020-02-03    0.800006
RKDA    2020-06-12    0.586174
SOHU    2020-06-12    0.443688
SPRT    2020-06-12    0.556976
SRRA    2020-06-12    0.333683
TMED    2020-03-20    0.943100
TOCA    2020-06-12    0.902666
UMRX    2020-06-12    0.448187
WSTL    2020-06-12    0.438776
Name: P/NetNet, dtype: float64

In [23]:
# Oldest date that is allowed for a row.
date_limit = datetime.now() - timedelta(days=30)

# Load the latest share-prices from the data-hub.
df_prices_latest = hub.load_shareprices(variant='latest')

# Boolean mask for the tickers that satisfy this condition.
mask_date_limit = (df_prices_latest.reset_index(DATE)[DATE] > date_limit)

# Show the result.
mask_date_limit.head()

Ticker
A        True
AA       True
AAC     False
AAL      True
AAMC     True
Name: Date, dtype: bool

In [24]:
mask &= mask_date_limit

In [25]:
df_signals.loc[mask, P_NETNET]

Ticker  Date      
AAMC    2020-06-12    0.783820
ALT     2020-06-12    0.933848
AVGR    2020-06-12    0.145914
CGA     2020-06-12    0.058775
CLBS    2020-06-12    0.471950
CLRB    2020-06-12    0.561256
CRVS    2020-06-12    0.960585
CUO     2020-05-19    0.580236
CYCC    2020-06-12    0.202383
CYIG    2020-06-12    0.050105
FPRX    2020-06-12    0.939102
GLYC    2020-06-12    0.702256
GTXI    2020-06-12    0.515962
KKR     2020-06-12    0.689317
NLNK    2020-06-12    0.593927
NSPR    2020-06-12    0.142141
RKDA    2020-06-12    0.586174
SOHU    2020-06-12    0.443688
SPRT    2020-06-12    0.556976
SRRA    2020-06-12    0.333683
TOCA    2020-06-12    0.902666
UMRX    2020-06-12    0.448187
WSTL    2020-06-12    0.438776
Name: P/NetNet, dtype: float64

# Screener for Many Criteria

In [38]:
mask = (df_signals[MARKET_CAP] > 1e9)

In [39]:
mask &= (df_signals[CURRENT_RATIO] > 2)
mask &= (df_signals[DEBT_RATIO] < 0.5)
mask &= (df_signals[SALES_GROWTH_YOY] > 0.1)

In [40]:
mask &= mask_date_limit

In [42]:
columns = [PFCF, PE, ROA, ROE, CURRENT_RATIO, DEBT_RATIO,MARKET_CAP]
df_signals.loc[mask, columns].sort_values(by=PFCF, ascending=True)

,,P/FCF,P/E,Return on Assets,Return on Equity,Current Ratio,Debt Ratio,Market-Cap
Ticker,Date,,,,,,,
TDOC,2020-06-12,-4038.521105,-113.531153,-0.066622,-0.103211,7.542175,0.146290,1.173594e+10
TWLO,2020-06-12,-794.710685,-146.149892,-0.032699,-0.040488,6.030387,0.108056,1.968975e+10
BGNE,2020-06-12,-175.343592,-166.144879,-0.339196,-0.454072,7.523711,0.039784,1.224177e+11
KEM,2020-06-12,-133.335093,7.781812,0.156731,0.323088,2.305620,0.223407,1.607621e+09
HALO,2020-06-12,-90.536903,-64.699858,-0.119021,-0.196399,2.949878,0.245637,3.304416e+09
...,...,...,...,...,...,...,...,...
RGEN,2020-06-12,180.017768,253.010554,0.026597,0.033954,2.180545,0.133487,5.369390e+09
FIVE,2020-06-12,326.486569,40.500559,0.096405,0.241357,2.863692,0.090188,6.216957e+09
RNG,2020-06-12,414.230899,-704.296624,-0.030905,-0.090105,3.081805,0.434058,2.101973e+10


In [43]:
df_signals.loc[mask, columns].sort_values(by=PFCF, ascending=True).head(10)

,,P/FCF,P/E,Return on Assets,Return on Equity,Current Ratio,Debt Ratio,Market-Cap
Ticker,Date,,,,,,,
TDOC,2020-06-12,-4038.521105,-113.531153,-0.066622,-0.103211,7.542175,0.146290,1.173594e+10
TWLO,2020-06-12,-794.710685,-146.149892,-0.032699,-0.040488,6.030387,0.108056,1.968975e+10
BGNE,2020-06-12,-175.343592,-166.144879,-0.339196,-0.454072,7.523711,0.039784,1.224177e+11
KEM,2020-06-12,-133.335093,7.781812,0.156731,0.323088,2.305620,0.223407,1.607621e+09
HALO,2020-06-12,-90.536903,-64.699858,-0.119021,-0.196399,2.949878,0.245637,3.304416e+09
ALG,2020-06-12,-85.411555,16.040870,0.087012,0.142202,4.630242,0.211637,1.189527e+09
EXAS,2020-06-12,-42.071545,-48.195826,-0.123569,-0.281597,8.392850,0.462654,1.054448e+10
RDFN,2020-06-12,-39.667226,-39.556492,-0.126554,-0.231332,4.729556,0.226740,2.876509e+09
SRPT,2020-06-12,-19.029304,-26.800161,-0.205305,-0.298983,12.167662,0.352352,1.080577e+10
